In [3]:
!pip install kaggle

In [9]:
!kaggle competitions download -c santander-customer-satisfaction


  0%|          | 0.00/8.25M [00:00<?, ?B/s]
 12%|#2        | 1.00M/8.25M [00:04<00:29, 257kB/s]
 24%|##4       | 2.00M/8.25M [00:12<00:32, 200kB/s]
 36%|###6      | 3.00M/8.25M [00:19<00:30, 182kB/s]
 49%|####8     | 4.00M/8.25M [00:25<00:25, 177kB/s]
 61%|######    | 5.00M/8.25M [00:30<00:18, 184kB/s]
 73%|#######2  | 6.00M/8.25M [00:35<00:12, 192kB/s]
 85%|########4 | 7.00M/8.25M [00:41<00:07, 183kB/s]
 97%|#########7| 8.00M/8.25M [00:50<00:01, 158kB/s]
100%|##########| 8.25M/8.25M [00:52<00:00, 143kB/s]


In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

path = "./res/santander-customer-satisfaction/"
test = pd.read_csv(path+"test.csv")
train = pd.read_csv(path+"train.csv")

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [14]:
train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,...,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,88.89,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.00,0.18,3.00,2.07,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,195.0,195.0,0.0,0.0,195.0,0.0,0.0,195.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,27,3,18,48,0,186.09,0.00,91.56,138.84,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,...,3,6,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [20]:
train.isna().sum()

ID                               0
var3                             0
var15                            0
imp_ent_var16_ult1               0
imp_op_var39_comer_ult1          0
imp_op_var39_comer_ult3          0
imp_op_var40_comer_ult1          0
imp_op_var40_comer_ult3          0
imp_op_var40_efect_ult1          0
imp_op_var40_efect_ult3          0
imp_op_var40_ult1                0
imp_op_var41_comer_ult1          0
imp_op_var41_comer_ult3          0
imp_op_var41_efect_ult1          0
imp_op_var41_efect_ult3          0
imp_op_var41_ult1                0
imp_op_var39_efect_ult1          0
imp_op_var39_efect_ult3          0
imp_op_var39_ult1                0
imp_sal_var16_ult1               0
ind_var1_0                       0
ind_var1                         0
ind_var2_0                       0
ind_var2                         0
ind_var5_0                       0
ind_var5                         0
ind_var6_0                       0
ind_var6                         0
ind_var8_0          

In [19]:
train.isna().sum().sum()

0

In [21]:
train["TARGET"].value_counts()

0    73012
1     3008
Name: TARGET, dtype: int64

In [96]:
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split

X = train.iloc[:,:-1]
y = train.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)

print("accuracy Score : {}".format(accuracy_score(y_test, y_pred)))
print("ROC-AUC Score : {}".format(roc_auc_score(y_test, y_pred)))

accuracy Score : 0.9607340173638517
ROC-AUC Score : 0.5023498810161343


In [27]:
train.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,...,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.0,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.0,76020.000000,76020.000000,7.602000e+04,7.602000e+04,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,68.803937,113.056934,68.205140,113.225058,137.242763,68.618087,113.792410,140.403479,5.

In [68]:
var = train["var3"]
var_mean = train["var3"].mean()
var_std = train["var3"].std()

lower_bound_idx = var < var_mean - 3* var_std
upper_bound_idx = var > var_mean + 3* var_std

var[lower_bound_idx] = var.mean()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,...,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET


In [56]:
num_over_th = 0
num_outlier_th = 100
columns = train.columns
for col in columns:
    var = train[col]
    var_mean = train[col].mean()
    var_std = train[col].std()

    lower_bound_idx = var < var_mean - 3* var_std
    upper_bound_idx = var > var_mean + 3* var_std

    num_outliers = lower_bound_idx.sum() + upper_bound_idx.sum()
    if num_outliers > num_outlier_th:
        num_over_th += 1
        print("  columns : {}".format(col))
        print("    -> num of oulier : {}\n".format(num_outliers))

print("num over th : {}".format(num_over_th))

  columns : var3
    -> num of oulier : 116

  columns : var15
    -> num of oulier : 1245

  columns : imp_ent_var16_ult1
    -> num of oulier : 220

  columns : imp_op_var39_comer_ult1
    -> num of oulier : 1416

  columns : imp_op_var39_comer_ult3
    -> num of oulier : 1403

  columns : imp_op_var40_comer_ult1
    -> num of oulier : 200

  columns : imp_op_var40_comer_ult3
    -> num of oulier : 234

  columns : imp_op_var40_ult1
    -> num of oulier : 170

  columns : imp_op_var41_comer_ult1
    -> num of oulier : 1486

  columns : imp_op_var41_comer_ult3
    -> num of oulier : 1459

  columns : imp_op_var41_efect_ult1
    -> num of oulier : 753

  columns : imp_op_var41_efect_ult3
    -> num of oulier : 579

  columns : imp_op_var41_ult1
    -> num of oulier : 1098

  columns : imp_op_var39_efect_ult1
    -> num of oulier : 756

  columns : imp_op_var39_efect_ult3
    -> num of oulier : 573

  columns : imp_op_var39_ult1
    -> num of oulier : 1082

  columns : ind_var1_0
    ->

  columns : num_var45_hace2
    -> num of oulier : 1383

  columns : num_var45_hace3
    -> num of oulier : 1556

  columns : num_var45_ult1
    -> num of oulier : 1406

  columns : num_var45_ult3
    -> num of oulier : 1461

  columns : saldo_medio_var5_hace2
    -> num of oulier : 763

  columns : saldo_medio_var5_hace3
    -> num of oulier : 567

  columns : saldo_medio_var5_ult1
    -> num of oulier : 546

  columns : saldo_medio_var5_ult3
    -> num of oulier : 665

  columns : saldo_medio_var8_hace2
    -> num of oulier : 196

  columns : saldo_medio_var8_ult1
    -> num of oulier : 311

  columns : saldo_medio_var8_ult3
    -> num of oulier : 310

  columns : saldo_medio_var12_hace2
    -> num of oulier : 883

  columns : saldo_medio_var12_hace3
    -> num of oulier : 436

  columns : saldo_medio_var12_ult1
    -> num of oulier : 1067

  columns : saldo_medio_var12_ult3
    -> num of oulier : 1047

  columns : saldo_medio_var13_corto_hace2
    -> num of oulier : 1190

  columns 

In [60]:
num_over_th = 0
num_outlier_th = 100
total_outlier = 0

columns = X.columns
for col in columns:
    var = X[col]
    var_mean = X[col].mean()
    var_std = X[col].std()

    lower_bound_idx = var < var_mean - 10* var_std
    upper_bound_idx = var > var_mean + 10* var_std

    num_outliers = lower_bound_idx.sum() + upper_bound_idx.sum()
    total_outlier += num_outliers
    if num_outliers > num_outlier_th:
        num_over_th += 1
        print("  columns : {}".format(col))
        print("    -> num of oulier : {}\n".format(num_outliers))

print("num over th : {}".format(num_over_th))
print("total_outlier : {}".format(total_outlier))

  columns : var3
    -> num of oulier : 116

  columns : imp_op_var39_comer_ult1
    -> num of oulier : 113

  columns : imp_op_var39_comer_ult3
    -> num of oulier : 124

  columns : imp_op_var41_comer_ult1
    -> num of oulier : 109

  columns : imp_op_var41_comer_ult3
    -> num of oulier : 112

  columns : ind_var1
    -> num of oulier : 286

  columns : ind_var14
    -> num of oulier : 403

  columns : ind_var17_0
    -> num of oulier : 137

  columns : ind_var17
    -> num of oulier : 110

  columns : ind_var19
    -> num of oulier : 319

  columns : ind_var20_0
    -> num of oulier : 276

  columns : ind_var20
    -> num of oulier : 205

  columns : ind_var30_0
    -> num of oulier : 343

  columns : ind_var31_0
    -> num of oulier : 325

  columns : ind_var31
    -> num of oulier : 279

  columns : ind_var40
    -> num of oulier : 283

  columns : ind_var39
    -> num of oulier : 283

  columns : ind_var44_0
    -> num of oulier : 143

  columns : ind_var44
    -> num of ouli

In [ ]:
X.drop("ID", inplace=True)

In [130]:
columns = X.columns
for col in columns:
    var = X[col]
    var_mean = X[col].mean()
    var_std = X[col].std()

    lower_bound_idx = var < var_mean - 10* var_std
    upper_bound_idx = var > var_mean + 10* var_std
    X.loc[lower_bound_idx, col] = X[col].mean()
    X.loc[upper_bound_idx, col] = X[col].mean()

ID 75964.05072349381
var3 -1523.1992765061825
var15 33.21286503551697
imp_ent_var16_ult1 86.20826519337021
imp_op_var39_comer_ult1 72.36306708760878
imp_op_var39_comer_ult3 119.52963220205162
imp_op_var40_comer_ult1 3.5591298342541426
imp_op_var40_comer_ult3 6.472697711128651
imp_op_var40_efect_ult1 0.41294632991318075
imp_op_var40_efect_ult3 0.567352407261247
imp_op_var40_ult1 3.1607154696132596
imp_op_var41_comer_ult1 68.8039372533545
imp_op_var41_comer_ult3 113.05693449092325
imp_op_var41_efect_ult1 68.20514048934491
imp_op_var41_efect_ult3 113.22505761641676
imp_op_var41_ult1 137.2427632202048
imp_op_var39_efect_ult1 68.6180868192581
imp_op_var39_efect_ult3 113.79241002367803
imp_op_var39_ult1 140.4034786898179
imp_sal_var16_ult1 5.4776756116811365
ind_var1_0 0.011457511181268087
ind_var1 0.0037621678505656408
ind_var2_0 0.0
ind_var2 0.0
ind_var5_0 0.9580242041568009
ind_var5 0.6637595369639568
ind_var6_0 0.00010523546435148645
ind_var6 2.6308866087871613e-05
ind_var8_0 0.032833464

imp_trasp_var33_in_hace3 2.7897722967640095
imp_trasp_var33_in_ult1 0.3147012628255722
imp_trasp_var33_out_hace3 0.0
imp_trasp_var33_out_ult1 0.03946329913180742
imp_venta_var44_hace3 3.7871369376479875
imp_venta_var44_ult1 81.43382794001577
ind_var7_emit_ult1 3.9463299131807416e-05
ind_var7_recib_ult1 0.00269665877400684
ind_var10_ult1 0.08087345435411734
ind_var10cte_ult1 0.09215995790581426
ind_var9_cte_ult1 0.09686924493554327
ind_var9_ult1 0.08591160220994475
ind_var43_emit_ult1 0.06658774006840305
ind_var43_recib_ult1 0.12930807682188897
var21 32.54932912391476
num_var2_0_ult1 0.0
num_var2_ult1 0.0
num_aport_var13_hace3 0.07588792423046567
num_aport_var13_ult1 0.017955801104972375
num_aport_var17_hace3 0.0015390686661404893
num_aport_var17_ult1 0.003393843725335438
num_aport_var33_hace3 0.0010655090765588004
num_aport_var33_ult1 0.00031570639305445933
num_var7_emit_ult1 0.00011838989739542226
num_var7_recib_ult1 0.010299921073401736
num_compra_var44_hace3 0.0018547750591949488
nu

In [138]:
X.describe()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,...,num_sal_var16_ult1,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.0,76020.0,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.0,76020.000000,76020.000000,7.602000e+04,7.602000e+04,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04
mean,2.713431,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,68.803937,113.056934,68.205140,113.225058,137.242763,68.618087,113.792410,140.403479,5.477

In [157]:
X = train.iloc[:,:-1]
X.drop(columns="ID", inplace=True)
X["var3"].replace(-999999, 0, inplace= True)

In [158]:
columns = X.columns
for col in columns:
    var = X[col]
    var_mean = X[col].mean()
    var_std = X[col].std()

    lower_bound_idx = var < var_mean - 5* var_std
    upper_bound_idx = var > var_mean + 5* var_std
    X.loc[lower_bound_idx, col] = X[col].mode()
    X.loc[upper_bound_idx, col] = X[col].mode()

In [159]:
X.describe()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,...,num_sal_var16_ult1,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,75691.000000,75999.000000,75891.000000,75377.000000,75416.000000,75863.000000,75837.000000,75990.000000,75985.000000,75881.000000,75349.000000,75407.000000,75725.000000,75778.000000,75611.000000,75726.000000,75775.000000,75604.000000,75983.000000,75149.0,75734.0,76020.0,76020.0,76020.000000,76020.000000,76012.0,76018.0,73524.0,73846.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,75247.0,75260.0,76018.0,76018.0,76020.000000,74222.0,75617.0,75883.0,75910.0,76018.0,76018.0,75701.0,75744.0,75815.0,76020.000000,73140.0,74011.0,74147.0,73925.0,74147.0,74223.0,74223.0,76020.0,76020.0,76020.0,76020.0,...,75924.0,75470.000000,75667.000000,75411.000000,76018.0,76016.0,76020.0,76016.0,76014.0,76015.0,76020.0,76019.0,76017.0,75976.0,75417.000000,75450.000000,75418.000000,75479.000000,76020.0,75607.000000,75722.000000,75706.000000,75671.000000,75901.000000,75979.000000,75845.000000,75837.000000,75520.000000,75722.000000,75422.000000,75432.000000,75327.000000,75672.000000,75204.000000,75216.000000,75700.000000,75890.000000,75697.000000,75696.000000,76018.0,76020.0,76018.0,76018.0,76010.000000,76015.000000,75996.000000,75997.000000,76017.000000,76019.0,76017.0,76017.0,75986.000000,76000.000000,75986.000000,75985.000000,75971.000000,75994.000000,75965.000000,75959.000000,7.590800e+04
mean,2.133873,33.194226,43.806216,48.905982,83.335684,0.393347,0.701615,0.009635,0.014374,0.265138,46.235591,79.118856,46.747929,82.048086,103.519599,46.956063,82.131385,105.348518,0.642489,0.0,0.0,0.0,0.0,0.958024,0.663760,0.0,0.0,0.0,0.0,0.067522,0.045462,0.052249,0.042936,0.041476,0.0,0.0,0.0,0.0,0.050855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.042370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)

print("accuracy Score : {}".format(accuracy_score(y_test, lgbm.predict(X_test))))
print("ROC-AUC Score : {}".format(roc_auc_score(y_test, lgbm.predict_proba(X_test)[:, 1])))

accuracy Score : 0.9598789792159957
ROC-AUC Score : 0.8421026245908767


In [ ]:
roc_auc_score()

In [164]:
X = train.iloc[:,:-1]

X.drop(columns="ID", inplace=True)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)

print("before replace outlier")
print("accuracy Score : {}".format(accuracy_score(y_test, lgbm.predict(X_test))))
print("ROC-AUC Score : {}\n".format(roc_auc_score(y_test, lgbm.predict_proba(X_test)[:, 1])))


X.describe()

before replace outlier
accuracy Score : 0.9632333596421995
ROC-AUC Score : 0.8345281581059178



,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,...,num_sal_var16_ult1,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.0,76020.0,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.0,76020.000000,76020.000000,7.602000e+04,7.602000e+04,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04
mean,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,68.803937,113.056934,68.205140,113.225058,137.242763,68.618087,113.792410,140.403479,5

In [165]:
columns = X.columns
for col in columns:
    var = X[col]
    var_mean = X[col].mean()
    var_std = X[col].std()

    lower_bound_idx = var < var_mean - 5* var_std
    upper_bound_idx = var > var_mean + 5* var_std
    X.loc[lower_bound_idx, col] = X[col].mode()
    X.loc[upper_bound_idx, col] = X[col].mode()



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
print("after replace outlier")
print("accuracy Score : {}".format(accuracy_score(y_test, lgbm.predict(X_test))))
print("ROC-AUC Score : {}".format(roc_auc_score(y_test, lgbm.predict_proba(X_test)[:, 1])))

X.describe()

after replace outlier
accuracy Score : 0.9607340173638517
ROC-AUC Score : 0.822154755529428


,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,...,num_sal_var16_ult1,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,75904.000000,75999.000000,75891.000000,75377.000000,75416.000000,75863.000000,75837.000000,75990.000000,75985.000000,75881.000000,75349.000000,75407.000000,75725.000000,75778.000000,75611.000000,75726.000000,75775.000000,75604.000000,75983.000000,75149.0,75734.0,76020.0,76020.0,76020.000000,76020.000000,76012.0,76018.0,73524.0,73846.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,75247.0,75260.0,76018.0,76018.0,76020.000000,74222.0,75617.0,75883.0,75910.0,76018.0,76018.0,75701.0,75744.0,75815.0,76020.000000,73140.0,74011.0,74147.0,73925.0,74147.0,74223.0,74223.0,76020.0,76020.0,76020.0,76020.0,...,75924.0,75470.000000,75667.000000,75411.000000,76018.0,76016.0,76020.0,76016.0,76014.0,76015.0,76020.0,76019.0,76017.0,75976.0,75417.000000,75450.000000,75418.000000,75479.000000,76020.0,75607.000000,75722.000000,75706.000000,75671.000000,75901.000000,75979.000000,75845.000000,75837.000000,75520.000000,75722.000000,75422.000000,75432.000000,75327.000000,75672.000000,75204.000000,75216.000000,75700.000000,75890.000000,75697.000000,75696.000000,76018.0,76020.0,76018.0,76018.0,76010.000000,76015.000000,75996.000000,75997.000000,76017.000000,76019.0,76017.0,76017.0,75986.000000,76000.000000,75986.000000,75985.000000,75971.000000,75994.000000,75965.000000,75959.000000,7.590800e+04
mean,2.717577,33.194226,43.806216,48.905982,83.335684,0.393347,0.701615,0.009635,0.014374,0.265138,46.235591,79.118856,46.747929,82.048086,103.519599,46.956063,82.131385,105.348518,0.642489,0.0,0.0,0.0,0.0,0.958024,0.663760,0.0,0.0,0.0,0.0,0.067522,0.045462,0.052249,0.042936,0.041476,0.0,0.0,0.0,0.0,0.050855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.042370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [166]:
from sklearn.model_selection import cross_val_score

X = train.iloc[:,:-1]
X.drop(columns="ID", inplace=True)
lgbm = LGBMClassifier()

scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("before replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))

before replace outlier
ROC-AUC Score : 0.832546087614436



In [167]:
columns = X.columns
for col in columns:
    var = X[col]
    var_mean = X[col].mean()
    var_std = X[col].std()

    lower_bound_idx = var < var_mean - 5* var_std
    upper_bound_idx = var > var_mean + 5* var_std
    X.loc[lower_bound_idx, col] = X[col].mode()
    X.loc[upper_bound_idx, col] = X[col].mode()



lgbm = LGBMClassifier()
scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("after replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))

after replace outlier
ROC-AUC Score : 0.831254504095701



,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,...,num_sal_var16_ult1,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,75904.000000,75999.000000,75891.000000,75377.000000,75416.000000,75863.000000,75837.000000,75990.000000,75985.000000,75881.000000,75349.000000,75407.000000,75725.000000,75778.000000,75611.000000,75726.000000,75775.000000,75604.000000,75983.000000,75149.0,75734.0,76020.0,76020.0,76020.000000,76020.000000,76012.0,76018.0,73524.0,73846.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,75247.0,75260.0,76018.0,76018.0,76020.000000,74222.0,75617.0,75883.0,75910.0,76018.0,76018.0,75701.0,75744.0,75815.0,76020.000000,73140.0,74011.0,74147.0,73925.0,74147.0,74223.0,74223.0,76020.0,76020.0,76020.0,76020.0,...,75924.0,75470.000000,75667.000000,75411.000000,76018.0,76016.0,76020.0,76016.0,76014.0,76015.0,76020.0,76019.0,76017.0,75976.0,75417.000000,75450.000000,75418.000000,75479.000000,76020.0,75607.000000,75722.000000,75706.000000,75671.000000,75901.000000,75979.000000,75845.000000,75837.000000,75520.000000,75722.000000,75422.000000,75432.000000,75327.000000,75672.000000,75204.000000,75216.000000,75700.000000,75890.000000,75697.000000,75696.000000,76018.0,76020.0,76018.0,76018.0,76010.000000,76015.000000,75996.000000,75997.000000,76017.000000,76019.0,76017.0,76017.0,75986.000000,76000.000000,75986.000000,75985.000000,75971.000000,75994.000000,75965.000000,75959.000000,7.590800e+04
mean,2.717577,33.194226,43.806216,48.905982,83.335684,0.393347,0.701615,0.009635,0.014374,0.265138,46.235591,79.118856,46.747929,82.048086,103.519599,46.956063,82.131385,105.348518,0.642489,0.0,0.0,0.0,0.0,0.958024,0.663760,0.0,0.0,0.0,0.0,0.067522,0.045462,0.052249,0.042936,0.041476,0.0,0.0,0.0,0.0,0.050855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.042370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [168]:

X = train.iloc[:,:-1]
X.drop(columns="ID", inplace=True)
lgbm = LGBMClassifier()

scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("before replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))


columns = X.columns
for col in columns:
    var = X[col]
    var_mean = X[col].mean()
    var_std = X[col].std()

    lower_bound_idx = var < var_mean - 100* var_std
    upper_bound_idx = var > var_mean + 100* var_std
    X.loc[lower_bound_idx, col] = X[col].mode()
    X.loc[upper_bound_idx, col] = X[col].mode()



lgbm = LGBMClassifier()
scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("after replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))

before replace outlier
ROC-AUC Score : 0.832546087614436

after replace outlier
ROC-AUC Score : 0.8319947132989114



In [ ]:

X = train.iloc[:,:-1]
X.drop(columns="ID", inplace=True)
lgbm = LGBMClassifier()

scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("before replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))


columns = X.columns
for col in columns:
    var = X[col]
    var_mean = X[col].mean()
    var_std = X[col].std()

    lower_bound_idx = var < var_mean - 100* var_std
    upper_bound_idx = var > var_mean + 100* var_std
    X.loc[lower_bound_idx, col] = X[col].mode()
    X.loc[upper_bound_idx, col] = X[col].mode()



lgbm = LGBMClassifier()
scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("after replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))

In [170]:
X.describe()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,...,num_sal_var16_ult1,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,76020.000000,76020.000000,76019.000000,76020.000000,76020.000000,76020.000000,76020.000000,76019.000000,76018.000000,76020.000000,76020.000000,76020.000000,76020.000000,76018.000000,76020.000000,76020.000000,76018.000000,76020.000000,76018.000000,76020.000000,76020.000000,76020.0,76020.0,76020.000000,76020.000000,76020.000000,76018.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76018.0,76018.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76018.0,76018.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.0,76020.0,...,76020.000000,76020.000000,76020.000000,76020.000000,76018.0,76016.0,76020.0,76016.0,76014.0,76019.000000,76020.0,76019.0,76017.0,76017.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,76019.000000,76020.000000,76020.000000,76019.000000,76018.000000,76019.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76019.000000,7.602000e+04,7.602000e+04,76018.0,76020.0,76019.000000,76019.000000,76019.000000,76019.000000,76019.000000,76019.000000,76018.000000,76019.0,76019.000000,76017.0,76019.000000,76018.000000,76019.000000,76019.000000,76019.000000,76017.000000,76019.000000,76018.000000,7.601900e+04
mean,-1523.199277,33.212865,83.446932,72.363067,119.529632,3.559130,6.472698,0.326131,0.430426,3.160715,68.803937,113.056934,68.205140,110.229142,137.242763,68.618087,110.796509,140.403479,3.332959,0.011458,0.003762,0.0,0.0,0.958024,0.663760,0.000105,0.0,0.032

In [171]:
train.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,...,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.0,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.000000,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,7.602000e+04,76020.000000,76020.0,76020.000000,76020.000000,7.602000e+04,7.602000e+04,7.602000e+04,7.602000e+04,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,68.803937,113.056934,68.205140,113.225058,137.242763,68.618087,113.792410,140.403479,5.

In [172]:

X = train.iloc[:,:-1]
X.drop(columns="ID", inplace=True)
lgbm = LGBMClassifier()

scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("before replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))


columns = X.columns
for col in columns:
    var = X[col]
    var_mean = X[col].mean()
    var_std = X[col].std()

    lower_bound_idx = var < var_mean - 15* var_std
    upper_bound_idx = var > var_mean + 15* var_std
    X.loc[lower_bound_idx, col] = X[col].mode()
    X.loc[upper_bound_idx, col] = X[col].mode()



lgbm = LGBMClassifier()
scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("after replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))

before replace outlier
ROC-AUC Score : 0.832546087614436

after replace outlier
ROC-AUC Score : 0.831791926010968



In [173]:
X.describe()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,...,num_sal_var16_ult1,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_trasp_var17_in_hace3,num_trasp_var17_in_ult1,num_trasp_var17_out_hace3,num_trasp_var17_out_ult1,num_trasp_var33_in_hace3,num_trasp_var33_in_ult1,num_trasp_var33_out_hace3,num_trasp_var33_out_ult1,num_venta_var44_hace3,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_var2_ult1,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
count,75904.000000,76020.000000,75985.000000,75979.000000,75983.000000,75963.000000,75964.000000,76000.000000,75995.000000,75977.000000,75984.000000,75987.000000,75983.000000,76001.000000,75983.000000,75983.000000,76002.000000,75982.000000,76010.000000,76020.000000,75734.0,76020.0,76020.0,76020.000000,76020.000000,76012.0,76018.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76018.0,76018.0,76020.000000,76020.000000,76020.000000,75883.0,75910.0,76018.0,76018.0,75701.0,75744.0,75815.0,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.0,76020.0,76020.0,76020.0,...,75924.0,75983.000000,75982.000000,75955.000000,76018.0,76016.0,76020.0,76016.0,76014.0,76015.0,76020.0,76019.0,76017.0,76000.000000,75999.000000,76005.000000,75988.000000,75997.000000,76020.0,75961.000000,75984.000000,75947.000000,75969.000000,75987.000000,75998.000000,75983.000000,75981.000000,75976.000000,75943.000000,75975.000000,75974.000000,75957.000000,75928.000000,76020.000000,75994.000000,75937.000000,75975.000000,75945.000000,75941.000000,76018.0,76020.0,76018.0,76018.0,76017.000000,76019.000000,76013.000000,76014.000000,76017.000000,76019.0,76017.0,76017.0,75992.000000,76004.000000,75996.000000,75994.000000,76002.000000,76005.000000,75997.000000,75994.000000,7.599200e+04
mean,2.717577,33.212865,59.972890,68.890750,114.128145,1.480083,2.990620,0.051476,0.058673,1.343867,65.883611,108.418885,60.661096,103.582333,128.695037,60.973642,104.337925,131.558688,2.063253,0.011458,0.0,0.0,0.0,0.958024,0.663760,0.0,0.0,0.032833,0.028598,0.067522,0.045462,0.052249,0.042936,0.041476,0.010168,0.00999

In [175]:
X = train.iloc[:,:-1]
X.drop(columns="ID", inplace=True)
X["var3"].replace(-999999, X["var3"].mode(), inplace=True)
lgbm = LGBMClassifier()

scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("before replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))


columns = X.columns
for col in columns:
    var = X[col]
    var_mean = X[col].mean()
    var_std = X[col].std()

    lower_bound_idx = var < var_mean - 200* var_std
    upper_bound_idx = var > var_mean + 200* var_std
    X.loc[lower_bound_idx, col] = X[col].mode()
    X.loc[upper_bound_idx, col] = X[col].mode()



lgbm = LGBMClassifier()
scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("after replace outlier")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))

before replace outlier
ROC-AUC Score : 0.832546087614436

after replace outlier
ROC-AUC Score : 0.8320502107547607



In [176]:
X = train.iloc[:,:-1]
X.drop(columns="ID", inplace=True)
lgbm = LGBMClassifier()

scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))

ROC-AUC Score : 0.832546087614436



In [ ]:
X = train.iloc[:,:-1]
lgbm = LGBMClassifier()

scores = cross_val_score(lgbm, X, y, cv= 5, scoring="roc_auc")
print("ROC-AUC Score : {}\n".format(np.mean(scores)))